In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
import os


In [5]:
df = pd.read_csv(r'C:\Users\ss6365\Desktop\location_privacy_final\collected\machine_learning\attack1\laplace\200\merged_laplace_0.1_encoded_200.csv')
df

,longitude,latitude,identifier,perturbed_latitude,perturbed_longitude,lat_bin,lon_bin,location_id
0,-77.680333,43.083838,1.0,43.083840,-77.680344,97,66,19467
1,-77.680991,43.083803,1.0,43.083827,-77.681023,97,62,19463
2,-77.681017,43.083802,1.0,43.083831,-77.680987,97,62,19463
3,-77.681042,43.083802,1.0,43.083767,-77.681051,97,62,19463
4,-77.681090,43.083802,1.0,43.083803,-77.681167,97,62,19463
...,...,...,...,...,...,...,...,...
55420,-77.680445,43.083879,46.0,43.083866,-77.680440,98,65,19666
55421,-77.680442,43.083868,46.0,43.083876,-77.680448,98,65,19666
55422,-77.680441,43.083863,46.0,43.083831,-77.680385,98,65,19666
55423,-77.680442,43.083860,46.0,43.083824,-77.680429,98,65,19666


In [6]:
df['identifier'].nunique()

36

In [7]:
features

NameError: name 'features' is not defined

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np



def knn_rule(data , strategy_ln = True, weights_f = 'uniform', metrics = 'manhattan', points_per_trace = 1):

    features = data[['perturbed_latitude', 'perturbed_longitude']]
    labels = data['location_id']

    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    features_scaled_df = pd.DataFrame(features_scaled, columns=['perturbed_latitude', 'perturbed_longitude'])

    X_train, X_test, y_train, y_test, identifiers_train, identifiers_test = train_test_split(
                                            features_scaled_df, labels, data['identifier'], test_size=0.3, random_state=None
                                            )

    if strategy_ln:
        k = round(np.log(len(X_train)))
    else:
        k = round(np.log10(len(X_train)))
    #weights_f = 'distance', 'uniform'
    #metrics = manhattan, ‘euclidean’, ‘haversine’, ‘cosine’

    knn = KNeighborsClassifier(n_neighbors=k, weights = weights_f, metric=metrics, n_jobs=-1)    

    knn.fit(X_train, y_train)

    
    
    # Create a DataFrame for easier manipulation
    test_df = pd.DataFrame(X_test, columns=['perturbed_latitude', 'perturbed_longitude'])
    test_df['identifier'] = identifiers_test.values
    test_df['true_label'] = y_test.values
    
    trace_accuracies = []
    
    for identifier, group in test_df.groupby('identifier'):
        # Select a subset of the group based on points_per_trace
        subset = group.head(points_per_trace)
        # Predict the subset
        y_pred = knn.predict(subset[['perturbed_latitude', 'perturbed_longitude']])
        # Calculate accuracy for this subset
        accuracy = np.mean(y_pred == subset['true_label'])
        trace_accuracies.append(accuracy)

    overall_trace_accuracy = np.mean(trace_accuracies)

    return 1-overall_trace_accuracy

# Load the data



In [20]:
data =  pd.read_csv(r'C:\Users\ss6365\Desktop\location_privacy_final\collected\machine_learning\attack1\laplace\200\merged_laplace_0.1_encoded_200.csv')
knn_rule(data , strategy_ln = True, weights_f = 'uniform', metrics = 'manhattan', points_per_trace = 7)

0.24206349206349187

In [28]:
def all_directory_knn(file_directory, strategy_ln = True, weights_f = 'uniform', metrics = 'manhattan', num_runs = 1, points_per_trace = 1):

    all_files = os.listdir(file_directory)
    csv_files = [file for file in all_files if file.endswith('.csv')]

    bayes_risks = [0] * len(csv_files)
    
    for _ in range(num_runs):

        for idx, file_name in enumerate(csv_files):

            file_path = os.path.join(file_directory, file_name)
            data = pd.read_csv(file_path)

            bayes_risk = knn_rule(data , strategy_ln = True, weights_f = 'uniform', metrics = 'manhattan', points_per_trace = 1)
            bayes_risks[idx] += bayes_risk
            

    bayesRisk_avg = [x / num_runs for x in bayes_risks]

    return bayesRisk_avg

In [37]:
file_directory = r'C:\Users\ss6365\Desktop\picasso\data'
a=all_directory_knn(file_directory, strategy_ln = True, weights_f = 'distance', metrics = 'euclidean', num_runs = 100, points_per_trace = 1)
a

[0.2397222222222223]

In [38]:
file_directory = r'C:\Users\ss6365\Desktop\picasso\data'
a=all_directory_knn(file_directory, strategy_ln = True, weights_f = 'distance', metrics = 'euclidean', num_runs = 100, points_per_trace = 2)
a

[0.22611111111111107]

In [39]:
file_directory = r'C:\Users\ss6365\Desktop\picasso\data'
a=all_directory_knn(file_directory, strategy_ln = True, weights_f = 'distance', metrics = 'euclidean', num_runs = 100, points_per_trace = 3)
a

[0.2236111111111111]

In [40]:
file_directory = r'C:\Users\ss6365\Desktop\picasso\data'
a=all_directory_knn(file_directory, strategy_ln = True, weights_f = 'distance', metrics = 'euclidean', num_runs = 100, points_per_trace = 10)
a

[0.20499999999999993]

In [49]:
def bayes_risk_trace(file_directory, strategy_ln = True, weights_f = 'distance', metrics = 'euclidean', num_runs = 1, trace = 10):

    bayes_trace = []

    for i in range(trace):
        bayes = all_directory_knn(file_directory, strategy_ln = True, weights_f = 'distance', metrics = 'euclidean', num_runs = 1, points_per_trace = i+1)
        
        bayes_trace.append(bayes)
    flattened_list = [item for sublist in bayes_trace for item in sublist]    
    return flattened_list 

In [53]:
file_directory = r'C:\Users\ss6365\Desktop\picasso\data'
bayes_risk_trace(file_directory, strategy_ln = True, weights_f = 'distance', metrics = 'euclidean', num_runs = 20, trace = 5)

[0.3055555555555556,
 0.2222222222222222,
 0.2222222222222222,
 0.2777777777777778,
 0.2222222222222222]

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.preprocessing import StandardScaler
# from itertools import islice

# def sliding_window(seq, n=2):
#     "Returns a sliding window (of width n) over data from the iterable"
#     it = iter(seq)
#     result = tuple(islice(it, n))
#     if len(result) == n:
#         yield result
#     for elem in it:
#         result = result[1:] + (elem,)
#         yield result

# def knn_rule(data, strategy_ln=True, weights_f='uniform', metrics='manhattan', points_per_trace=1):
#     # Prepare features and labels
#     features = data[['perturbed_latitude', 'perturbed_longitude']]
#     labels = data['location_id']

#     # Scale the features
#     scaler = StandardScaler()
#     features_scaled = scaler.fit_transform(features)
#     features_scaled_df = pd.DataFrame(features_scaled, columns=['perturbed_latitude', 'perturbed_longitude'])

#     # Split the data into training and testing sets
#     X_train, X_test, y_train, y_test, identifiers_train, identifiers_test = train_test_split(
#         features_scaled_df, labels, data['identifier'], test_size=0.3, random_state=None
#     )

#     # Determine k using logarithmic strategy
#     if strategy_ln:
#         k = round(np.log(len(X_train)))
#     else:
#         k = round(np.log10(len(X_train)))

#     # Initialize and train the k-NN classifier
#     knn = KNeighborsClassifier(n_neighbors=k, weights=weights_f, metric=metrics, n_jobs=-1)
#     knn.fit(X_train, y_train)

#     # Create a DataFrame for easier manipulation of test data
#     test_df = pd.DataFrame(X_test, columns=['perturbed_latitude', 'perturbed_longitude'])
#     test_df['identifier'] = identifiers_test.values
#     test_df['true_label'] = y_test.values

#     # Evaluate trace accuracy using a sliding window approach
#     trace_accuracies = []
#     for identifier, group in test_df.groupby('identifier'):
#         group_list = group.to_dict('records')  # Convert group to list of dicts for sliding window
#         for subset in sliding_window(group_list, points_per_trace):
#             subset_df = pd.DataFrame(subset)  # Convert each window to DataFrame
#             y_pred = knn.predict(subset_df[['perturbed_latitude', 'perturbed_longitude']])
#             accuracy = np.mean(y_pred == subset_df['true_label'])
#             trace_accuracies.append(accuracy)

#     # Calculate and return the overall trace-based error rate
#     overall_trace_accuracy = np.mean(trace_accuracies)
#     return 1 - overall_trace_accuracy

# # Load the data
# data = pd.read_csv(r'C:\Users\ss6365\Desktop\location_privacy_final\collected\machine_learning\attack1\laplace\200\merged_laplace_0.1_encoded_200.csv')
# # Run the KNN rule with specific parameters
# error_rate = knn_rule(data, strategy_ln=True, weights_f='uniform', metrics='manhattan', points_per_trace=7)
# print(f"Overall Trace-Based Error Rate: {error_rate}")
